# I-LOFAR XST Tutorial 

This tutorial is based on the tutorial from Michiel Brentjens [here](https://github.com/brentjens/lofarimaging/tree/master).

It is aimed at familiarising the users with array covariance or crosslets (XST) data produced by the internation LOFAR station like the Irish Lofarstaion or I-LOFAR. XTS data is the receiver cross correlation averaged over a specified time period written out as 64-bit little-endian floats.


If the station has $N_{rcu}$ RCUs, an $N_{rcu} × N_{rcu}$ complex array covariance matrix will be produced from each integration period. Denoting the covariance of receivers i and j with xi,j, the data points of single integration period are written to the file in the order


$Re(x_{0,0}), Im(x_{0,0}), Re(x_{0,1}), Im(x_{0,1}), Re(x{0,2}), Im(x_{0,2}), . . . , Re(x_{N_{rcu},N_{rcu}}), Im(x_{N_{rcu},N_{rcu}} )$, where $Re$ and $Im$ denote the real and imaginary parts of the complex data values.

In [ ]:
import astropy.units as u
import numpy as np

from lofarantpos.db import LofarAntennaDatabase
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

Let look at the folders and see can we find some data

In [ ]:
ls

In [ ]:
!ls XST_data/

This data was taken by I-LOFAR and can be found here https://data.lofar.ie/2017/12/25/xst/

## Exercise 1: Inspecting and loading an XST file

Lets load in the data as before `np.fromfile` is useful here

In [ ]:
# np.complex128 as 64bit for real and 64bit for imag

International stations have 192 RCUS the sizme should be $N_{rcu} \times  N_{rcu}$ so $N_{rcu} = \sqrt{size}$ 

Lets rehshape the data to have the correct format of $N_{rcu} \times N_{rcu}$ 

## Not sure about this need to double check

The data contains both polarisations so extract the x and y polarisations


## Exercise 2: Make all sky image

Now that we have the complex visiblty data we need to compute some other values to make the all sky images.

First we need the positions of the antenna for the Irish station IE613 and since looking at mode 3 will be the LBA

Next we need to compute the baselines or vectors between the each antenna pair

Lets convert the subband to a frequency this obsevation was take in subband 194.

Remember the conversion from subband number to frequency from earlier

$$\nu = (n-1+ \frac{s}{512})\frac{\nu_{clock}}{2},$$

where $n$ is the nyquist zone, $s$ is the subband number and $\nu_{clock}$ is the clock frequency.

In this example $n=1$ and $\nu_{clock} = 200$ MHz.

In [ ]:
# Convert subband to frequncei in Hz !!

print(freq)

The function below computes the inverse Fourier transform of station obaservations

In [ ]:
# Define a function to do the imaging
def sky_imager(visibilities,baselines,freq,im_x,im_y):
    img = np.zeros([im_y,im_x])
    for l_ix,l in enumerate(np.linspace(1,-1,im_y)):
        for m_ix,m in enumerate(np.linspace(-1,1,im_x)):
            n = np.sqrt(1-l**2-m**2)
            img[m_ix,l_ix] = np.mean(visibilities*np.exp(-2j*np.pi*freq*(baselines[:,:,0]*l + baselines[:,:,1]*m)/299792458.0))
    return img

Lets make image of the x polarisation

Now plot the imge using `plt.imshow`

What about the y polarisation

Again plot the imge using `plt.imshow`

## Exercise 3: Make a ground map/image

Using the same data but a different transform we can make a local ground map/image, this can be useful to locate local RFI.

The function below compute such a map.

In [ ]:
def ground_imager(visibilities,baselines,freq,im_x,im_y,dims,station_pqr,height=1.5):
    img = np.zeros([im_y,im_x])
    for p_ix,p in enumerate(np.linspace(dims[0],dims[1],im_x)):
        for q_ix,q in enumerate(np.linspace(dims[2],dims[3],im_y)):
            r = height
            pqr = np.array([p,q,r])
            antdist = np.linalg.norm(station_pqr - pqr[np.newaxis,:],axis=1)
            groundbase = np.zeros([96,96])
            for i in range(0,96):
                groundbase[i] = antdist[i] - antdist[:]
            # Note: this is RFI integration second - normal second, to take out interference
            img[q_ix,p_ix] = np.mean(visibilities*np.exp(-2j*np.pi*freq*(-groundbase)/299792458.0))
    return img

Let compute the ground image for the x polaristaion